In [4]:
import sys

from pathlib import Path

sys.path.append(str(Path("..").resolve()))

import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Tuple
from src.numerical_gem_pipeline import load_data, get_full_binary_grid,coefficients,get_inverse_metric,build_tetrads,check_orthonormality,calculate_field_strength_tensor,calculate_B3_via_eq8_rigorous,calculate_shift_curl_gem

DATA_FILE = "data.npz"

In [5]:
def main(data_path: str = DATA_FILE, run_name: str = "BBH_NONSpinning_test", use_git_folder: bool = False):
    """
    Args:
        data_path: Path to the .npz data file.
        run_name: Name of the subfolder (e.g., 'test_run_01').
        use_git_folder: If True, saves to 'results/' (Tracked). 
                        If False, saves to 'runs/' (Local/Ignored).
    """
    
    # 1. Determine Output Root
    # 'results' is for GIT (Final, Public)
    # 'runs' is for LOCAL (Diagnostics, Ignored)
    root_dir = "results" if use_git_folder else "runs"
    
    # 2. Construct Paths
    base_dir = os.path.join(root_dir, run_name, "figures")
    out_png = os.path.join(base_dir, "Inspiral_Fields_Refinementlevel_3.png")

    # 3. Create Directory
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        print(f"   📂 Created output directory: {base_dir}")

    print(f"   🖼️  Target: {out_png}")

    raw_data = load_data(data_path)
    # ... (Rest of the architecture remains invariant) ...
    data = get_full_binary_grid(raw_data)
    x, y = data["x"], data["y"]
    dx, dy = float(x[1] - x[0]), float(y[1] - y[0])

    print(f"Grid Shape: {data['alp'].shape}")

    # 1. Build & Verify Full Tetrads
    theta = build_tetrads(data)
    check_orthonormality(theta, data)

    # 2. Calculate Fields (Raw, no filtering)
    F_tensor = calculate_field_strength_tensor(theta, dx, dy)
    E_fields = gem_fields(theta, F_tensor)
    
    # 3. Calculate Magnetic Fields (Both versions)
    B3_x, B3_y = calculate_B3_via_eq8_rigorous(theta, dx, dy)
    Bz_GEM = calculate_shift_curl_gem(data["betax"], data["betay"], dx, dy)

    # --- INSERTED LOGIC: CALCULATE q0 (Eq 25) ---
    # q0 = 0.5 * sqrt(gamma) * (E^2 + B^2)
    Ex_raw, Ey_raw = E_fields["0"]
    det_gamma = data['gxx'] * data['gyy'] - data['gxy']**2
    sqrt_gamma = np.sqrt(np.abs(det_gamma))
    E_sq = Ex_raw**2 + Ey_raw**2
    B_sq = B3_x**2 + B3_y**2
    q0 = 0.5 * sqrt_gamma * (E_sq + B_sq)
    # --------------------------------------------

    # 4. Plotting Prep
    def prep(arr):
        if arr.shape == (len(y), len(x)): return arr
        return arr.T

    vals_x, vals_y = x, y
    vals_alp = prep(data['alp'])
    vals_Ex, vals_Ey = prep(Ex_raw), prep(Ey_raw)
    vals_Bz_GEM = prep(Bz_GEM)
    vals_betax, vals_betay = prep(data['betax']), prep(data['betay'])
    vals_B3x, vals_B3y = prep(B3_x), prep(B3_y)
    vals_q0 = prep(q0)
    Shift_mag = np.sqrt(vals_betax**2 + vals_betay**2)

    # --- NEW: Toroidal Magnetic Field Calculation ---
    X_grid, Y_grid = np.meshgrid(vals_x, vals_y) 
    R_grid = np.sqrt(X_grid**2 + Y_grid**2)
    R_grid[R_grid < 1e-6] = 1.0 # Avoid division by zero
    B_toroidal = (-Y_grid * vals_B3x + X_grid * vals_B3y) / R_grid
    # ------------------------------------------------

    # --- PLOTTING: 2x3 Grid (Landscape for Slides) ---
    # Changed from (3, 2) to (2, 3) to create a wide layout
    # Increased width to 24, reduced height to 13 to match 16:9 slide ratio
    fig, axes = plt.subplots(2, 3, figsize=(24, 13)) 

    sim_time = raw_data.get("time", None)
    if sim_time is not None:
        main_title = f"BBH-GEM Analysis | Simulation Time: t = {float(sim_time):.2f} M"
    else:
        main_title = "BBH-GEM Analysis | Time: Unknown"
    fig.suptitle(main_title, fontsize=20, fontweight='bold', y=0.98)
    
    # --- PLOT 1: Magnetic Field (Structure) ---
    # Position: Top Left
    ax = axes[0, 0]
    ax.streamplot(vals_x, vals_y, vals_B3x, vals_B3y, color="black", density=1.2, arrowsize=1.0)
    ax.set_title("1. Magnetic Field Lines (Structure)", fontsize=14)
    ax.set_aspect("equal")

    # --- PLOT 2: Electric Field (Structure) ---
    # Position: Top Middle
    ax = axes[0, 1]
    ax.streamplot(vals_x, vals_y, vals_Ex, vals_Ey, color="black", density=1.2, arrowsize=1.0)
    ax.set_title("2. Electric Field Lines (Structure)", fontsize=14)
    ax.set_aspect("equal")

    # --- PLOT 3: Paper Replica (Shift with Lapse Colorbar) ---
    # Position: Top Right
    ax = axes[0, 2]
    pcm3 = ax.pcolormesh(vals_x, vals_y, vals_alp, cmap="Reds_r", shading="auto", vmin=0.0, vmax=1.0)
    ax.streamplot(vals_x, vals_y, vals_betax, vals_betay, color="black", density=1.2, arrowsize=1.0)
    ax.set_title("3. Shift Streamlines on Lapse", fontsize=14)
    ax.set_aspect("equal")
    fig.colorbar(pcm3, ax=ax, label=r"Lapse ($\alpha$)")

    # --- PLOT 4: Shift Magnitude ---
    # Position: Bottom Left
    ax = axes[1, 0]
    pcm4 = ax.pcolormesh(vals_x, vals_y, Shift_mag, cmap="viridis", shading="auto")
    ax.streamplot(vals_x, vals_y, vals_betax, vals_betay, color="white", density=1.2, arrowsize=1.0)
    ax.set_title("4. Shift Magnitude", fontsize=14)
    ax.set_aspect("equal")
    fig.colorbar(pcm4, ax=ax, label=r"Shift Magnitude ($|\vec{\beta}|$)")

    # --- PLOT 5: Toroidal Magnetic Field (B_phi) ---
    # Position: Bottom Middle
    ax = axes[1, 1]
    limit_B = np.max(np.abs(B_toroidal)) * 0.8
    pcm5 = ax.pcolormesh(vals_x, vals_y, B_toroidal, cmap="RdBu_r", shading="auto", vmin=-limit_B, vmax=limit_B)
    ax.streamplot(vals_x, vals_y, vals_B3x, vals_B3y, color="black", density=1.0, arrowsize=0.8, linewidth=0.5)
    ax.set_title("5. Toroidal Magnetic Field (B^phi)", fontsize=14)
    ax.set_aspect("equal")
    fig.colorbar(pcm5, ax=ax, label="B_phi (Normalized by R)")

    # --- EMPTY SLOT ---
    # Position: Bottom Right (Hide it)
    axes[1, 2].axis('off')

    # Apply limits to all active plots
    for ax in axes.flat:
        if ax.lines or ax.collections: 
            ax.set_xlim(-12, 12)
            ax.set_ylim(-14, 14)
            ax.tick_params(labelsize=10)

    # Use tight_layout with padding to ensure labels don't overlap
    plt.tight_layout(rect=[0, 0.03, 1, 0.95], h_pad=3.0, w_pad=3.0)
    plt.savefig(out_png, dpi=150)
    print(f"🎉 Full Rigorous Output saved to {out_png}")

if __name__ == "__main__":
    main()

   🖼️  Target: runs/BBH_NONSpinning_test/figures/Inspiral_Fields_Refinementlevel_3.png


FileNotFoundError: Could not find data.npz